In [1]:
import torch
import torch.nn as nn
import torch.nn.init
import torchvision.models as models
from torch.nn.utils.clip_grad import clip_grad_norm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from torchtext.models import ROBERTA_BASE_ENCODER
from torchtext.functional import to_tensor

import numpy as np
import numpy.ma as ma
import pandas as pd
import glob
import os
from PIL import Image
from tqdm import tqdm
import warnings

/home/geoph/Documents/python-related/torchenv/lib64/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
warnings.filterwarnings('ignore')

### 1. Preparation of dataset

The unified dataset with rows for each pair of image names and corresponding texts is made.

In [3]:
def create_dataset(texts_path, images_path):
    """Create dataframe with path to image and corresponding text description"""

    texts_df = pd.read_csv(texts_path)
    texts_df['text'] = texts_df['color'] + " " + texts_df['name'] + " " + texts_df['description']
    texts_df = texts_df[['Unnamed: 0','text']]
    texts_df['product'] = np.arange(len(texts_df))

    df = pd.DataFrame(columns=["Image_id", "Image_name","Text","Product_id"])  
    
    for i, image in enumerate(glob.glob(images_path)):
        img_name = os.path.basename(image)
        key_img_name = img_name.split('_')[0]
        img_descr = texts_df[texts_df['Unnamed: 0']==int(key_img_name)].iloc[0,1:]
        df = df.append({'Image_id':i, 'Image_name': img_name, 'Text':img_descr[0], 'Product_id':img_descr[1]}, ignore_index=True)
    
    return df, df['Product_id'].unique()

The class CustomImageLoader helps to build custom loader of the training data in the form of the batches. It takes into account that all the images, corresponding to the same product have to be in the same batch. 

In [4]:
class CustomImageLoader:
    """Creation of batches of images, texts and their ids."""
    def __init__(self, annotations_file, img_dir, transform=None, batch_size=10):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        self.batch_size = batch_size

    def getbatch(self, prod_idx):
        batch = []
        sliced_indices = self.img_labels[self.img_labels['Product_id'].isin(prod_idx)].index

        # print(f"GETBATCH {prod_idx=}\n{sliced_indices=}\n\n{self.img_labels[self.img_labels['Product_id'].isin(prod_idx)]}")
        for i in sliced_indices:
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[i, 1])
            image = Image.open(img_path)
            image_id, text, product_id = self.img_labels.iloc[i, 0], self.img_labels.iloc[i, 2], self.img_labels.iloc[i, 3]
            if self.transform:
                image = self.transform(image)
            batch.append((image,text,(image_id, product_id)))
            
        unzipped = list(zip(*batch))
        
        return unzipped[0], unzipped[1], unzipped[2]
    
    def __getitem__(self, prod_idx):
        return self.getbatch(prod_idx)
    
    def __iter__(self):
        products_groups = [products[i:i + self.batch_size] for i in range(0, len(products), self.batch_size)]
        for i in products_groups:
            yield self.getbatch(i)

Preparation steps for passing data to dataloader are made, as well as, the constant variables for the whole program are defined.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transformation of images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224,224)),
    transforms.Normalize((0.5,), (0.5,))  # Scale images to [-1, 1]
])

# Data paths (Arina)
# descriptions_data = "./processed_data/processedSKUs_nodups.csv"
# images_folder = "./processed_data/images/*.jpg"
# img_dir = "./processed_data/images/"
# George
descriptions_data = "../processedSKUs_nodups.csv"
images_folder = "../images_onlyids/*.jpg"
img_dir = "../images_onlyids/"

# Creation of organized dataframe
annotations_file, products = create_dataset(descriptions_data, images_folder)

# Creation of true constant adjacency matrix, where rows - products, columns - images. 
ADJACENCY_TRUE = pd.crosstab(annotations_file.Product_id, annotations_file.Image_id)
# ADJACENCY_TRUE = np.zeros(ADJACENCY_TRUE.shape)
# for p_id, img_id in zip(annotations_file.Product_id, annotations_file.Image_id):
#     #print(f"{p_id=}\n{img_id=}")
#     ADJACENCY_TRUE[p_id, img_id] = 1
#     # for img_id in annotations_file[annotations_file.Product_id == p_id].Image_id:
#     #     ADJACENCY_TRUE[p_id, img_id]
# ADJACENCY_TRUE = pd.DataFrame(ADJACENCY_TRUE)
# ADJACENCY_TRUE.rename_axis("Image_id", "columns", inplace=True)
# ADJACENCY_TRUE.rename_axis("Product_id", "index", inplace=True)

# Creation of custom Batch Loader, where batch contains images, belonging to same product
dataset = CustomImageLoader(annotations_file, img_dir, transform=transform)

# Creation of batches of products. For each product there are 2/3 images, so the actual batch size is ~20-30 pairs.
prod_batch_size = 10
products_groups = [products[i:i + prod_batch_size] for i in range(0, len(products), prod_batch_size)]

In [6]:
annotations_file.Product_id.max(), annotations_file.Product_id.unique().shape

(238, (239,))

In [7]:
ADJACENCY_TRUE

Image_id,0,1,2,3,4,5,6,7,8,9,...,684,685,686,687,688,689,690,691,692,693
Product_id,,,,,,,,,,,,,,,,,,,,,
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
annotations_file[annotations_file.Product_id == 100]

,Image_id,Image_name,Text,Product_id
17,17,105_0..jpg,18 Kt Rose Gold First Steps Earrings Crafted w...,100
115,115,105_1..jpg,18 Kt Rose Gold First Steps Earrings Crafted w...,100
297,297,105_2..jpg,18 Kt Rose Gold First Steps Earrings Crafted w...,100


In [9]:
ADJACENCY_TRUE.iloc[100,:][ADJACENCY_TRUE.iloc[100,:]==1]

Image_id
17     1
115    1
297    1
Name: 100, dtype: int64

In [10]:
for _ in dataset:
    break

In [11]:
ADJACENCY_TRUE.sum(axis=1).value_counts()

3    216
2     23
dtype: int64

In [12]:
ADJACENCY_TRUE

Image_id,0,1,2,3,4,5,6,7,8,9,...,684,685,686,687,688,689,690,691,692,693
Product_id,,,,,,,,,,,,,,,,,,,,,
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Definition of classes for image, text encoders and supporting functions

In [13]:
def normalization(X):
    """L2-normalization of features columns"""
    norm = torch.pow(X, 2).sum(dim=1, keepdim=True).sqrt()
    X = torch.div(X, norm)
    
    return X

In [14]:
class ImageEncoder(nn.Module):
    def __init__(self, embedding_size, cnn_type):
        """Initializing parameters"""
        super(ImageEncoder, self).__init__()
        self.embedding_size = embedding_size # Size of projected image
        self.cnn = self.load_cnn(cnn_type)

        # No need to finetune parameters = frozen layers
        for param in self.cnn.parameters():
            param.requires_grad = False

        # Replacing last fully connected layer with new one
        self.fc = nn.Linear(self.cnn.classifier._modules['6'].in_features, embedding_size)
        self.cnn.classifier = nn.Sequential(*list(self.cnn.classifier.children())[:-1])

        # Initializing the weights of fully-connected layer, which makes projection to new space
        self.initialization_weights()
  
    def load_cnn(self, cnn_type):
        """Loading pretrained model"""
        model = models.__dict__[cnn_type](pretrained=True)

        return model

    def initialization_weights(self):
        """Xavier initialization"""
        r = np.sqrt(6.) / np.sqrt(self.fc.in_features + self.fc.out_features)
        self.fc.weight.data.uniform_(-r, r)
        self.fc.bias.data.fill_(0)

    def forward(self, X):
        """Creation of features"""
        # Creation of embeddings
        features = self.cnn(X)

        # Normalization of embeddings
        features = normalization(features)

        # Projection to new space
        features = self.fc(features)

        return features

In [15]:
ROBERTA_OUT_DIM = 768

class TextEncoder(nn.Module):

    def __init__(self, embedding_size):
        """Initializing parameters"""
        super(TextEncoder, self).__init__()
        self.embedding_size = embedding_size # Size of projected text
        self.roberta = ROBERTA_BASE_ENCODER.get_model()
        self.transform = ROBERTA_BASE_ENCODER.transform()

        # Linear layer
        self.fc = nn.Linear(ROBERTA_OUT_DIM, embedding_size)

        # Initializing the weights of fully-connected layer, which makes projection to new space
        self.initialization_weights()
        
    def _roberta_encode(self, batch):
        transformed = self.transform(batch)
        model_input = to_tensor(transformed, padding_value=1)
        return self.roberta(model_input)

    def initialization_weights(self):
        """Xavier initialization"""
        r = np.sqrt(6.) / np.sqrt(self.fc.in_features + self.fc.out_features)
        self.fc.weight.data.uniform_(-r, r)
        self.fc.bias.data.fill_(0)

    def forward(self, X, lengths=None):
        """Creation of features"""
        # Creation of embeddings
        features = self._roberta_encode(list(X))

        # Normalization of embeddings
        features = normalization(features)

        # Projection to new space
        features = self.fc(features)

        return features

### 3. Definition of loss class

In [16]:
def similarity_score(img, text):
    """Similarity calculation"""
    
    return text.mm(img.t())

In [17]:
ADJACENCY_TRUE.iloc[:10, :50]

Image_id,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
Product_id,,,,,,,,,,,,,,,,,,,,,
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
class CombinedLoss(nn.Module):
    """Class for combined loss"""

    def __init__(self, margin=0, lambda_coeff=0.1):
        super(CombinedLoss, self).__init__()
        self.margin = margin
        self.lambda_coeff = lambda_coeff
        # self.sim = similarity_score
    
    def _calc_penalty(self, similarities):
        if isinstance(similarities, torch.Tensor):
            similarities = similarities.detach().numpy()
        adj_tmp = np.zeros_like(similarities)
        adj_tmp[similarities.argmax(0), np.arange(similarities.shape[1])] = 1
        images_mapped = adj_tmp.sum(1)
        assert images_mapped.sum() == similarities.shape[1], f"{images_mapped.shape=}, {similarities.shape=}"
        penalty = self.lambda_coeff * len(np.where(images_mapped==0)[0])
        return penalty

    def forward(self, image_emb, label_emb, img_product_ids):
        # Image_emb - batch_size x embedding size
        # Label_emb - number of unique products from batch x embedding size
        # Img_product_ids - batch_size x 2, where in each row there is (image_index, product_index).
        batch_size = image_emb.shape[0]
        products_num = label_emb.shape[0]
        
        # Slicing from true adjacency matrix, prepared beforehand
        adj_true = ADJACENCY_TRUE.iloc[img_product_ids[:,1]]
        adj_true = adj_true.iloc[:,img_product_ids[:,0]]
        
        adj_true = adj_true[~adj_true.index.duplicated(keep="first")]
        # print(adj_true.sum(0).values)
        # Creation of similarity matrix of size - number of unique products from batch x batch_size
        sim_matrix = similarity_score(image_emb, label_emb)
        
        # Calculation of loss for each product
        # 1st - For each product minimum similarity with true images is calculated
        adj_true = torch.Tensor(adj_true.values)
        product_true = sim_matrix*adj_true
        product_min_sims = product_true.masked_fill(product_true == 0, np.inf).min(dim=1)[0]
        
        # 2nd - For each product maximum similarity score with image, not belonging to it
        adj_true_inv = torch.where(adj_true==0, 1, 0)
        product_false = sim_matrix*adj_true_inv
        product_max_sims, product_max_sims_indices = product_false.max(axis=1)
        
        # 3d - For each product maximum similarity between its images and products, other than the true one, is found
        not_product_max_sims = np.array([])
        for p in range(products_num):
            sim_p = (sim_matrix[np.arange(products_num)!=p,:]*adj_true[p,:]).max()
            not_product_max_sims = np.append(not_product_max_sims,sim_p.detach())
        not_product_max_sims = torch.Tensor(not_product_max_sims)
        
        # Caluculation of mean loss
        first_hinge = self.margin+not_product_max_sims - product_min_sims
        second_hinge = self.margin+product_max_sims - product_min_sims
        loss = torch.where(first_hinge<0,0,first_hinge)+torch.where(second_hinge<0,0,second_hinge)
        loss = torch.sum(loss)/products_num
        #Regularizer (this is just a single value)
        loss += self._calc_penalty(sim_matrix)
        return loss

### 4. Definition of class and functions for training

The JewelryClassifier class combines the procedure of creation of the embeddings for images and texts, as well as, calculates loss.

In [19]:
class JewelryClassifier:
    """Class, unifying the embeddings creation and loss calculation."""
    def __init__(self, emb_size=128, grad_clip=2, learning_rate=0.01, *loss_args):
        self.grad_clip = grad_clip
        self.im_enc = ImageEncoder(emb_size, 'vgg19')
        self.txt_enc = TextEncoder(emb_size)
        if torch.cuda.is_available():
            self.im_enc.cuda()
            self.txt_enc.cuda()
        self.criterion = CombinedLoss(*loss_args)
        params = list(self.txt_enc.fc.parameters())
        params += list(self.im_enc.fc.parameters())
        # The image cnn is fine-tuned but roberta is not.
        params += self.im_enc.cnn.parameters()
        self.params = params
        self.optimizer = torch.optim.Adam(params, lr=learning_rate)
        self.step = 0
    
    def state_dict(self):
        state_dict = [self.im_enc.state_dict(), self.txt_enc.state_dict()]
        return state_dict

    def load_state_dict(self, state_dict):
        self.im_enc.load_state_dict(state_dict[0])
        self.txt_enc.load_state_dict(state_dict[1])
    
    def save(self, path):
        torch.save(self.state_dict(), path)
    
    def on_stage_start(self, stage):
        if stage == "TRAIN":
            self.im_enc.train()
            self.txt_enc.train()
        elif stage == "VALID":
            self.im_enc.eval()
            self.txt_enc.eval()
    
    def train(self):
        return self.on_stage_start("TRAIN")
    
    def eval(self):
        return self.on_stage_start("VALID")
    
    # @staticmethod
    # def to_tensor(x, grad=True):
    #     if not isinstance(x, torch.Tensor):
    #         x = torch.Tensor(x, requires_grad=grad)
    #     if torch.cuda.is_available():
    #         x = x.cuda()
    #     return x
    
    def forward_emb(self, imgs, txts, lengths=None, grad=True):
        """Compute the image and text embeddings"""
        # Set mini-batch dataset
        #imgs = self.to_tensor(imgs, grad)
        #txts = self.to_tensor(txts, grad)

        # Forward
        imgs_emb = self.im_enc(imgs)
        txts_emb = self.txt_enc(txts, lengths)
        return imgs_emb, txts_emb

    def forward_loss(self, imgs_emb, txts_emb, img_product_ids):
        """Compute the loss given pairs of image and text embeddings and there ids"""
        loss = self.criterion(imgs_emb, txts_emb, img_product_ids)
        return loss

    def train_emb(self, imgs, txts, img_product_ids):
        """One training step given images and captions."""
        self.step += 1

        # compute the embeddings
        imgs_emb, txts_emb = self.forward_emb(imgs, txts)

        # measure accuracy and record loss
        self.optimizer.zero_grad()
        loss = self.forward_loss(imgs_emb, txts_emb.mean(axis=1), img_product_ids)

        # compute gradient and do SGD step
        loss.backward()
        if self.grad_clip > 0:
            clip_grad_norm(self.params, self.grad_clip)
        self.optimizer.step()
        return loss.item()

In [28]:
# That cycle has to be used to receive batches !!!
model = JewelryClassifier(128, 2, 0.01, 0.2, 0.1)
for i in tqdm(products_groups):
    X, Y, IDS = dataset.getbatch(i) # X=images, Y=descriptions of products, IDS=pairs of (image_id,product_id)
    images = torch.stack(X, dim=0) # Images are converted to batched tensor
    labels = pd.Series(Y).unique() # From descriptions of products only unique ones are  selected, preserving the order
    img_product_ids = np.array(IDS)
    loss = model.train_emb(images, labels, img_product_ids)
    
    break



  0%|                                                                                                                                                                                                                  | 0/24 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [26]:
def train(train_loader, val_loader, epochs, emb_size, grad_clip=2, use_valid=False, *loss_args):
    """The main training function."""
    model = JewelryClassifier(emb_size, grad_clip, *loss_args)
    pbar = tqdm(range(1, epochs+1))
    avg_loss = "inf"
    val_loss = "inf"
    def format_desc():
        pbar.set_description(f"AVG Loss={avg_loss}, Train Loss={loss}, Valid Loss={val_loss}  (epoch {epoch})")
    for epoch in range(1, epochs+1):
        model.train()
        # TODO: Add learning rate scheduler
        losses = []
        # pbar = tqdm(enumerate(train_loader))
        # for i, (im_batch, txt_batch, img_product_ids) in pbar:
        for i, pr_group in tqdm(enumerate(products_groups)):
            im_batch, txt_batch, img_product_ids = dataset.getbatch(pr_group)
            im_batch = torch.stack(im_batch, dim=0) # Images are converted to batched tensor
            txt_batch = pd.Series(txt_batch).unique() # From descriptions of products only unique ones are  selected, preserving the order
            img_product_ids = np.array(img_product_ids)
            loss = model.train_emb(im_batch, txt_batch, img_product_ids)
            losses.append(loss)
            format_desc()
        avg_loss = loss = sum(losses)/len(losses)
        format_desc()
        if use_valid:
            val_losses = []
            for i, (im_batch, txt_batch, txt_lengths) in enumerate(val_loader):
                model.eval()
                im_emb, txt_emb = model.forward_emb(im_batch, txt_batch, txt_lengths, grad=False)
                val_loss = model.forward_loss(im_emb, txt_emb)
                val_losses.append(val_loss)
                format_desc()
            val_loss = sum(val_losses)/len(val_losses)
            format_desc()
        # TODO Checkpointing
    # Save model
    model.save()

In [27]:
train(products_groups, None, 2, 128, 2, False, 0.2, 0.1)

  0%|                                                                                                                                                                                                                   | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]


AttributeError: 'CustomImageLoader' object has no attribute 'get_batch'